In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from scipy.optimize import minimize

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings('ignore')


In [121]:
mercado_df = pd.read_csv("C://Flavio/Projeto Otimizacao/Modelo Mercado/data_consolidado_final_jun24.csv",
                     encoding='latin-1',
                     delimiter=';')
mercado_df.shape

(96586, 36)

In [122]:
mercado_df = mercado_df[(~mercado_df.Grupo_veiculo_new.isin(['ARRIZO',
'Chery Passeio',   'Chery Tiggo'   , 'Erro'  ,'Kia Cerato','Kia Picanto' ,'Kia Sorento'  ,'Kia Soul' ,
'Kia Sportage'  ,'Rio']))]

mercado_df = mercado_df[(~mercado_df.fabrica.isin(['BYD','CAOA CHERY','GWM','KIA MOTORS','LEXUS']))]

mercado_df = mercado_df[(~mercado_df.idade_ult_RNS.isin(['_03 ou+ anos']))]
mercado_df.shape

(93838, 36)

In [123]:
df_mod=mercado_df[["premio_total_liq_cap","ano_mod_conv","classe_bonus",
                   "idade","is_casco","is_dmoral","is_rcdc","is_rcdm",
                   "RNS_Casco_calc",
                   "Grupo_veiculo_new", "fabrica","idade_ult_RNS","md_categoria",
                   "MD_comb","md_config","MD_franquia","md_menor26","md_sexo",
                   "MD_Subzona","md_tipo_renov","MD_uso_veic","uf"]]

df_mod.rename(columns={'premio_total_liq_cap':'premio_ofertado', 'ano_mod_conv': 'ano_mod'},inplace=True)

In [124]:
x_df_num = df_mod[['ano_mod', 'classe_bonus', 'idade', 'is_casco', 'is_dmoral', 'is_rcdc', 'is_rcdm', 'RNS_Casco_calc']]
x_df_num = x_df_num.fillna(0)
x_df_cat = df_mod[['Grupo_veiculo_new', 'fabrica', 'idade_ult_RNS', 'md_categoria', 'MD_comb', 'md_config', 'MD_franquia', 'md_menor26', 'md_sexo', 'MD_Subzona','md_tipo_renov', 'MD_uso_veic','uf']]
y   = df_mod[['premio_ofertado']]
#x_df_cat_ordinal = encoder.fit_transform(x_df_cat)
#x_df_cat_ordinal = pd.DataFrame(x_df_cat_ordinal)
x_df_dummy=pd.get_dummies(x_df_cat)
#x_df=pd.concat([x_df_num, x_df_cat_ordinal],axis=1)
x_df=pd.concat([x_df_num, x_df_dummy],axis=1)
x_train, x_test, y_train, y_test = train_test_split(x_df, y, test_size=0.2) 

In [125]:
x_df.shape

(93838, 519)

In [118]:
#x_df.columns.to_list()

In [126]:
x_df_num.columns[x_df_num.isnull().any()]

Index([], dtype='object')

In [127]:
x_df_cat.columns[x_df_cat.isnull().any()]

Index([], dtype='object')

%%time
from sklearn.ensemble import GradientBoostingRegressor
# inicia o modelo GBM
gbm = GradientBoostingRegressor(n_estimators=100,
                                max_leaf_nodes=32,
                                learning_rate=0.1, 
                                random_state=42)

# Treina o modelo
gbm.fit(x_train, y_train)

# previsão
y_pred = gbm.predict(x_test)

# Evaluate the model

print("Premio Estimado Médio: %.2f" % np.mean(y_pred) )
# checando a acurácia
mse = mean_squared_error(y_test, y_pred)
rmse = mse**(0.5)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % rmse) 

In [ ]:
#import pickle
# saving
#model_pkl_file = "mercado_model.pkl"  
#with open(model_pkl_file, 'wb') as file:  
#pickle.dump(gbm, file)

In [ ]:
# load model from pickle file
#with open(model_pkl_file, 'rb') as file:  
#model = pickle.load(file)

In [ ]:
#model = pickle.load(open("mercado_model", "rb"))

In [ ]:
#cols_when_model_builds = gbm.get_booster().feature_names

In [128]:
import lightgbm as lgb
params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'n_estimators':100,
    'num_leaves': 50,
    'min_depth': 5,
    'max_depth': 5,
    'learnnig_rage': 0.1,
    'metric': {'l2','l1'},
    'verbose': -1
}

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval  = lgb.Dataset(x_test, y_test, reference=lgb_train)

model_gbm = lgb.train(params,
                 train_set=lgb_train,
                 valid_sets=lgb_eval)

y_pred = model_gbm.predict(x_test)
print("Premio Estimado Médio: %.2f" % np.mean(y_pred) )
# accuracy check
mse = mean_squared_error(y_test, y_pred)
rmse = mse**(0.5)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % rmse) 

Premio Estimado Médio: 2621.76
MSE: 595044.21
RMSE: 771.39


## modelagem GLM

In [174]:
%%time
df_mod_mp_feb23may24_v3 = pd.read_csv("C://Flavio/Projeto Otimizacao/Modelo Renovacao/df_mod_mp_feb23may24_v3.csv",
                     encoding='latin-1',
                     delimiter='|')

df_mod_mp_feb23may24_v3.shape

CPU times: total: 6.06 s
Wall time: 7.19 s


(571594, 105)

In [175]:
df_mod_mp_feb23may24_v3.shape

(571594, 105)

In [176]:
#df_mod_mp_feb23may24_v3.columns.to_list()

In [177]:
df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[(~df_mod_mp_feb23may24_v3.Grupo_veiculo_new.isin(['BWM X',
'Blindados',
'Chevrolet Astra Sedan',
'Chrysler Passeio',
'Citroen Antigo',
'Citroen Xsara Picasso',
'Dodge Journey',
'Esportivo Porsche',
'Fiat ANTIGOS',
'Fiat Ducato',
'Fiat Ducato Furgao',
'Fiat Palio Antigo',
'Fiat Stilo',
'Ford Sedans Antigos',
'Hyundai Sonata',
'Hyundai Veloster',
'Hyundai Vera Cruz',
'IEV',
'JAC J2',
'JAC J3 Hatch',
'JAC J3 Sedan',
'JAC T',
'Kia Sportage Antigo',
'Lifan Passeio',
'Lifan X',
'MB Classe B',
'MB Linha E',
'MB Linha S',
'MB Sprinter Furgão',
'MB Sprinter Van',
'Nissan Tiida',
'Peugeot 106/205/206',
'Peugeot Hoggar',
'Pickup Nacional',
'Smart Fortwo',
'Troller T-4',
'VW Bora',
'VW Gol 1.8',
'VW Kombi',
'VW Parati',
'VW Passat',
'VW Santana',
'VW Saveiro Antigo',
'Van Importada',
]))]

df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[(~df_mod_mp_feb23may24_v3.FABRICA.isin(['CHERY',
'CHRYSLER','DEMAIS','DODGE','JAC MOTORS','KIA','LIFAN','PORSCHE','SMART','SSANGYONG','TROLLER']))]


df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[(~df_mod_mp_feb23may24_v3.idade_ult_RNS.isin([
'03 anos','04 anos','05 anos','06 anos','07 anos','08 anos','09 anos','10 ou +']))]


df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[(~df_mod_mp_feb23may24_v3.md_categoria.isin([
'Pickup Leve Importado']))]

df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[(~df_mod_mp_feb23may24_v3.MD_franquia.isin([
'ERRO']))]

df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[(~df_mod_mp_feb23may24_v3.MD_menor26.isin([
'ERR']))]

df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[(~df_mod_mp_feb23may24_v3.MD_sexo.isin([
'ERRO']))]


df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[(~df_mod_mp_feb23may24_v3.MD_Subzona.isin([
'88400-SERTAO PERNAMBUCANO']))]


df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[(~df_mod_mp_feb23may24_v3.MD_uso_veic.isin([
'Aluguel e Frete',
'Ambulância',
'Bar/Lanchonete',
'ERRO',
'Escolares',
'Funerárias',
'Lotações',
'Transporte de Carga',
'Transporte de Funcionarios',
'Turismos',
]))]


df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[(~df_mod_mp_feb23may24_v3.UF.isin([
'ND',]))]


df_mod_mp_feb23may24_v3.shape


(377345, 105)

In [ ]:
#sp = df_mod_mp_feb23may24_v3[df_mod_mp_feb23may24_v3['UF'] == 'SP']


In [ ]:
#hilux = sp[sp['Grupo_veiculo_new'].isin(["Toyota Hilux CD","Toyota Hilux CS","Toyota Hilux SW4"])]

In [ ]:
#sp_sem_hilux = sp[~sp['Grupo_veiculo_new'].isin(["Toyota Hilux CD","Toyota Hilux CS","Toyota Hilux SW4"])]


parte para incluir o preço do mercado na base de modelagem glm

In [178]:
colunas=['premio_ofertado','ano_mod','classe_bonus','MD_idade','is_casco_ofer','is_dmoral','is_rcdc','is_rcdm','RNS_Casco_calc',
 'Grupo_veiculo_new','FABRICA','idade_ult_RNS', 'md_categoria','MD_comb','MD_config','MD_franquia','MD_menor26','MD_sexo','MD_Subzona',  
 'MD_tipo_renov','MD_uso_veic','UF','per_com','Renovacao_ant','MD_est_civil','peso','premio_total','vigencia','premio_creserva',
        'anomes_dt_ter','teste_validacao']

In [179]:
df_escorar_gbm=df_mod_mp_feb23may24_v3[colunas]
#df_escorar_gbm = df_mod_mp_feb23may24_v3.copy()

In [180]:
df_escorar_gbm.shape

(377345, 31)

In [181]:
df_escorar_gbm = df_escorar_gbm[df_escorar_gbm['per_com'].notna()]
df_escorar_gbm = df_escorar_gbm[df_escorar_gbm['Renovacao_ant'].notna()]
df_escorar_gbm = df_escorar_gbm[df_escorar_gbm['MD_est_civil'].notna()]
df_escorar_gbm = df_escorar_gbm[df_escorar_gbm['peso'].notna()]
df_escorar_gbm = df_escorar_gbm.query('premio_total !=0')
df_escorar_gbm = df_escorar_gbm.query('vigencia == "Anual"')
df_escorar_gbm = df_escorar_gbm.query('premio_creserva >= 0')
df_escorar_gbm = df_escorar_gbm[~df_escorar_gbm['anomes_dt_ter'].isin([202310,202309,202308,202302,202402])]
df_escorar_gbm = df_escorar_gbm.query('is_casco_ofer > 0')
df_escorar_gbm = df_escorar_gbm[(df_escorar_gbm['per_com']>0) & (df_escorar_gbm['per_com']<50)]
df_escorar_gbm = df_escorar_gbm.query('premio_total >100')
df_escorar_gbm = df_escorar_gbm.query('MD_idade !="ERRO"')
df_escorar_gbm = df_escorar_gbm[(df_escorar_gbm.UF != "SP") | (~df_escorar_gbm.Grupo_veiculo_new.isin(["Toyota Hilux CD","Toyota Hilux CS","Toyota Hilux SW4"]))]
df_escorar_gbm = df_escorar_gbm[(df_escorar_gbm.premio_ofertado/df_escorar_gbm.premio_total <=2.3) &  (df_escorar_gbm.premio_ofertado/df_escorar_gbm.premio_total>=0.7)]
df_escorar_gbm.query('teste_validacao == "Trein"')
df_escorar_gbm.shape

(231857, 31)

In [182]:
df_escorar_gbm.rename(columns={'MD_idade':'idade', 
                                   'is_casco_ofer': 'is_casco',
                                   'FABRICA': 'fabrica',
                                   'MD_config':  'md_config',
                                   'MD_menor26': 'md_menor26',
                                   'MD_sexo': 'md_sexo',
                                   'UF': 'uf',
                                   'MD_tipo_renov':  'md_tipo_renov'},inplace=True)
df_escorar_gbm['idade'] = pd.to_numeric(df_escorar_gbm['idade'], errors='coerce')

In [183]:
df_num = df_escorar_gbm[['ano_mod', 'classe_bonus', 'idade', 'is_casco', 'is_dmoral', 'is_rcdc', 'is_rcdm', 'RNS_Casco_calc']]
x_df_num = x_df_num.fillna(0)
df_cat = df_escorar_gbm[['Grupo_veiculo_new', 'fabrica', 'idade_ult_RNS', 'md_categoria', 'MD_comb', 'md_config', 'MD_franquia', 'md_menor26', 'md_sexo', 'MD_Subzona','md_tipo_renov', 'MD_uso_veic','uf']]
df_cat_glm=pd.get_dummies(df_cat)
#df_cat_glm2=pd.get_dummies(df_cat)
df_cat_glm=pd.concat([df_num, df_cat_glm],axis=1)
#df_cat_glm2=pd.concat([df_escorar_gbm, df_cat_glm2],axis=1)

#GBM_mercado = gbm.predict(df_cat_glm, predict_disable_shape_check=True)

In [184]:
df_cat_glm2.shape

(231857, 616)

In [120]:
#df_cat_glm.columns.to_list()

In [140]:
#GBM_mercado = model_gbm.predict(df_cat_glm, predict_disable_shape_check=True)

In [186]:
GBM_mercado = model_gbm.predict(df_cat_glm)
np.average(GBM_mercado)

2571.7709049885116

In [188]:
df_escorar_gbm['GBM_mercado']=GBM_mercado
df_escorar_gbm.shape

(231857, 32)

#### prepara base de modelagem glm

In [189]:
df_mod_filtered=df_escorar_gbm[["premio_ofertado","premio_total",'GBM_mercado', 'premio_creserva','premio_demais_cob',"teste_validacao","ind_renovacao","MD_sexo","cod_corr",
                                "Cod_End","MD_sin_5a_cas","MD_tempo_casa_2","MD_idade_veic","MD_faixa_IScas",
                                "anomes_dt_ter" ,"MD_idade","per_com", "RNS_Casco_calc","MD_menor26","UF","Renovacao_ant",
                                "fx_ir_12m","is_casco_ofer","classe_bonus","MD_fator_ajuste","MD_capac","md_categoria",
                                "MD_comb","MD_qtd_assis_cpf", "idade_ult_RNS","canal","MD_tipo_renov",'FABRICA','MD_est_civil',
                                'peso','Grupo_veiculo_new','vigencia',
                                'Risco_PP_mod', 'Risco_PT_mod','Risco_RF_mod','Risco_DM_mod','Risco_AS_mod',
                                'Risco_DC_mod' ,'Risco_DMO_mod']]

KeyError: "['premio_demais_cob', 'ind_renovacao', 'MD_sexo', 'cod_corr', 'Cod_End', 'MD_sin_5a_cas', 'MD_tempo_casa_2', 'MD_idade_veic', 'MD_faixa_IScas', 'MD_idade', 'MD_menor26', 'UF', 'fx_ir_12m', 'is_casco_ofer', 'MD_fator_ajuste', 'MD_capac', 'MD_qtd_assis_cpf', 'canal', 'MD_tipo_renov', 'FABRICA', 'Risco_PP_mod', 'Risco_PT_mod', 'Risco_RF_mod', 'Risco_DM_mod', 'Risco_AS_mod', 'Risco_DC_mod', 'Risco_DMO_mod'] not in index"

In [14]:
df_mod_filtered = df_mod_filtered[df_mod_filtered['per_com'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['Renovacao_ant'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['MD_est_civil'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['peso'].notna()]
df_mod_filtered = df_mod_filtered.query('premio_total !=0')
df_mod_filtered = df_mod_filtered.query('vigencia == "Anual"')
df_mod_filtered = df_mod_filtered.query('premio_creserva >= 0')
df_mod_filtered = df_mod_filtered[~df_mod_filtered['anomes_dt_ter'].isin([202310,202309,202308,202302,202402])]
df_mod_filtered = df_mod_filtered.query('is_casco_ofer > 0')
df_mod_filtered = df_mod_filtered[(df_mod_filtered['per_com']>0) & (df_mod_filtered['per_com']<50)]
df_mod_filtered = df_mod_filtered.query('premio_total >100')
df_mod_filtered = df_mod_filtered.query('MD_idade !="ERRO"')
df_mod_filtered = df_mod_filtered[(df_mod_filtered.UF != "SP") | (~df_mod_filtered.Grupo_veiculo_new.isin(["Toyota Hilux CD","Toyota Hilux CS","Toyota Hilux SW4"]))]
df_mod_filtered = df_mod_filtered[(df_mod_filtered.premio_ofertado/df_mod_filtered.premio_total <=2.3) &  (df_mod_filtered.premio_ofertado/df_mod_filtered.premio_total>=0.7)]

df_mod_filtered.shape

NameError: name 'df_mod_filtered' is not defined

In [ ]:
df_mod_filtered.shape

In [ ]:
df_mod_filtered['classe_bonus'].describe()

In [ ]:
print(df_mod_filtered['classe_bonus'].value_counts(ascending=False))

In [ ]:
df_mod_filtered = df_mod_filtered[df_mod_filtered['per_com'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['Renovacao_ant'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['MD_est_civil'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['peso'].notna()]
df_mod_filtered.shape

In [ ]:
df_mod_filtered = df_mod_filtered.query('premio_total !=0')
df_mod_filtered.shape

In [ ]:
df_mod_filtered = df_mod_filtered.query('vigencia == "Anual"')
df_mod_filtered.shape

In [ ]:
df_mod_filtered = df_mod_filtered.query('premio_creserva >= 0')
df_mod_filtered.shape

In [ ]:
df_mod_filtered = df_mod_filtered[~df_mod_filtered['anomes_dt_ter'].isin([202310,202309,202308,202302,202402])]
df_mod_filtered.shape

In [ ]:
df_mod_filtered['DA_emissao'] =0.2349
df_mod_filtered['DA_producao'] =0.2749
df_mod_filtered['risco_dmais_cob'] =df_mod_filtered['premio_demais_cob']*0.2627*1.4
df_mod_filtered['premio_anterior'] =df_mod_filtered['premio_total']
df_mod_filtered['Act_cost']  = df_mod_filtered['Risco_PP_mod'] + df_mod_filtered['Risco_PT_mod'] + df_mod_filtered['Risco_RF_mod']+ df_mod_filtered['Risco_DM_mod']+ df_mod_filtered['Risco_AS_mod'] + df_mod_filtered['Risco_DC_mod'] +df_mod_filtered['Risco_DMO_mod'] + df_mod_filtered['risco_dmais_cob']
df_mod_filtered['Premio_IC100']  = df_mod_filtered['Act_cost']/(1-df_mod_filtered['per_com']/100-df_mod_filtered['DA_producao'] )
df_mod_filtered["premio"] = df_mod_filtered["premio_ofertado"]

In [ ]:
np.mean(df_mod_filtered['Premio_IC100'])

In [ ]:
df_mod=df_mod_filtered.copy()

In [ ]:
def categorizar(df):
    
    df['classe_bonus_bin'] = df['classe_bonus'].apply(lambda x: '00'  if x == 0 else 'Demais')   
    
    df['MD_sexo_bin'] = df['MD_sexo'].map({'Masculino': 'Masculino', 
                                 'Feminino': 'Feminino', 
                                  'ERRO': 'Masculino'}).astype(object)'
'

    df['fx_ir_12m_bin'] = df['fx_ir_12m'].apply(lambda x: '01 - 00-<=020' if x == '01 - 00-<=020' else
                                                              '02 - 20-<=040' if x == '02 - 20-<=040' else
                                                              '03 - 40-<=060' if x == '03 - 40-<=060' else
                                                              '04 - 60-<=080' if x == '04 - 60-<=080' else
                                                              '05 - 80-<=100' if x == '05 - 80-<=100' else
                                                              '02 - 20-<=040' if x == 'NI'           else 'ERROR')

  
    df['MD_comb_bin'] = df['MD_comb'].apply(lambda x: 'Gasolina' if x == 'Gasolina'  else 'Demais')

    df['canal_bin'] =  df['canal'].apply(lambda x: 'Corretor Mais' if x == 'Corretor Mais'  else 'Demais')
   
    df['per_com_bin'] = df['per_com'].apply(lambda x: '<10' if x < 10 else
                                                          '>=10' if x >= 10 else 'ERROR')    



    df['idade_ult_RNS_bin'] = df['idade_ult_RNS'].apply(lambda x: 'Sem Sinistro' if x == 'Sem Sinistro' else
                                                                      '01-05 ano' if x == '01 ano' else
                                                                      '01-05 ano' if x == '01 ano' else
                                                                      '01-05 ano' if x == '01 ano' else
                                                                      '01-05 ano' if x == '01 ano' else
                                                                      '01-05 ano' if x == '01 ano' else '06+ ano +')


    df['UF_bin'] = df['UF'].apply(lambda x: 'AL,CE,RS'  if x == 'AL' else
                                                'AL,CE,RS'  if x == 'CE' else
                                                'AL,CE,RS'  if x == 'RS' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'AP' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'DF' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'ES' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'PA' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'PB' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'RN' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'SC' else 
                                                'GO,MA,MG'             if x == 'GO' else 
                                                'GO,MA,MG'             if x == 'MA' else 
                                                'GO,MA,MG'             if x == 'MG' else 
                                                'MT,PR'                if x == 'MT' else
                                                'MT,PR'                if x == 'PR' else 
                                                'RJ'                   if x == 'RJ' else  'Demais')

    df['md_tipo_renov_ant_bin'] = df['Renovacao_ant'].apply(lambda x:     '1 - SeguroNovo'       if x == '1 - Seguro Novo'  else
                                                                          '2 - RenovPropria'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                          '2 - RenovPropria'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                          '3 - RenovCongenere'   if x == '4 - Renov Congenere sem sinistro' else
                                                                          '3 - RenovCongenere'   if x == '5 - Renov Congenere com sinistro' else
                                                                          '3 - RenovCongenere'   if x == '6 - Renov BB sem sinistro' else
                                                                          '3 - RenovCongenere'   if x == '7 - Renov BB com sinistro' else  '2 - RenovPropria')
       
     
    df['Cod_End_bin'] = df['Cod_End'].apply(lambda x: '13509'  if x == 13509 else
                                                      '13509'  if x == 50007 else
                                                      '13509'  if x == 50100 else 'Demais')   

    df['MD_capac_bin'] = df['MD_capac'].apply(lambda x: '5'  if x == 5  else 'Demais')  



    df['fator_ajuste_bin'] = df['MD_fator_ajuste'].apply(lambda x: '<101'    if x < 101 else
                                                                    '>=101'   if x >= 101  else '<101')   

    df['MD_qtd_assis_cpf_bin'] = df['MD_qtd_assis_cpf'].apply(lambda x: '00'  if x == 0 else
                                                                        '01'  if x == 1 else
                                                                        '>=2' if x >= 2 else 'ERROR')
     
    return df

In [ ]:
df_mod = categorizar(df_mod)

In [ ]:
df_train = df_mod.query('teste_validacao == "Trein"')
df_train.shape

In [ ]:
df_train = df_train.query('is_casco_ofer > 0')
df_train.shape

In [ ]:
df_train = df_train.query('premio_total >100')
df_train = df_train.query('MD_idade !="ERRO"')
df_train.shape

In [ ]:
df_train = df_train[(df_train['per_com']>0) & (df_train['per_com']<50)]
df_train.shape

In [ ]:
df_train= df_train[(df_train.UF != "SP") | (~df_train.Grupo_veiculo_new.isin(["Toyota Hilux CD","Toyota Hilux CS","Toyota Hilux SW4"]))]
df_train.shape

In [ ]:
df_train = df_train[(df_train.premio_ofertado/df_train.premio_total <=2.3) &  (df_train.premio_ofertado/df_train.premio_total>=0.7)]
df_train.shape

In [ ]:
print(df_train['idade_ult_RNS_bin'].value_counts(ascending=False))

In [ ]:
display(df_train[["is_casco_ofer","premio","Act_cost", "Premio_IC100", "per_com","ind_renovacao",'GBM_mercado']].describe().T)

In [ ]:
#df_train.columns.to_list()

In [ ]:
import statsmodels.formula.api as smf
#Modelo GLM de retenção
formula = 'ind_renovacao ~ 1 + np.log(premio) + np.divide(premio, is_casco_ofer) + np.log(np.divide(premio, GBM_mercado)) + np.log(np.divide(premio, Premio_IC100)) + (np.divide(premio, premio_anterior)+1) + C(md_tipo_renov_ant_bin,Treatment(reference="2 - RenovPropria")) + C(classe_bonus_bin,Treatment(reference="Demais")) + C(canal_bin,Treatment(reference="Demais")) + C(Cod_End_bin,Treatment(reference="Demais")) + C(fator_ajuste_bin,Treatment(reference="<101")) + per_com + C(per_com_bin,Treatment(reference=">=10")) + np.log(MD_tempo_casa_2+1) + C(idade_ult_RNS_bin,Treatment(reference="Sem Sinistro")) + C(MD_comb_bin,Treatment(reference="Demais")) + C(UF_bin,Treatment(reference="Demais")) + C(fx_ir_12m_bin,Treatment(reference="02 - 20-<=040")) + C(MD_tipo_renov,Treatment(reference="2 - Renov MAPFRE sem sinistro"))' 
model = smf.logit(formula , data=df_train).fit()
model.summary()

In [ ]:
df_train["prob_est"]=model.predict(df_train)

In [ ]:
import statsmodels.formula.api as smf
#Modelo GLM de retenção
formula = 'ind_renovacao ~ 1 + np.log(premio) + np.divide(premio, is_casco_ofer)  + np.log(np.divide(premio, Premio_IC100)) + (np.divide(premio, premio_anterior)+1) + C(md_tipo_renov_ant_bin,Treatment(reference="2 - RenovPropria")) + C(classe_bonus_bin,Treatment(reference="Demais")) + C(canal_bin,Treatment(reference="Demais")) + C(Cod_End_bin,Treatment(reference="Demais")) + C(fator_ajuste_bin,Treatment(reference="<101")) + per_com + C(per_com_bin,Treatment(reference=">=10")) + np.log(MD_tempo_casa_2+1) + C(idade_ult_RNS_bin,Treatment(reference="Sem Sinistro")) + C(MD_comb_bin,Treatment(reference="Demais")) + C(UF_bin,Treatment(reference="Demais")) + C(fx_ir_12m_bin,Treatment(reference="02 - 20-<=040")) + C(MD_tipo_renov,Treatment(reference="2 - Renov MAPFRE sem sinistro"))' 
#formula = 'ind_renovacao ~ 1 ' 
model_teste = smf.logit(formula , data=df_train).fit()
model_teste.summary()

### teste da elasticidade

In [ ]:
import statsmodels.formula.api as smf
#Modelo GLM de retenção
formula = 'ind_renovacao ~ 1 + np.log(premio)' 
model_elast = smf.logit(formula , data=df_train).fit()
model_elast.summary()

In [ ]:
price_elasticity = logreg.coef_[:, 0] * price_range * (1 - purchase_pr)

In [ ]:
model_elast

## Base Para escorar e otimizar

In [ ]:
df_lote_092024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/Table 7 Base Case Total  - teste novo.csv",
                     encoding='latin-1',
                     delimiter='|')

In [ ]:
df_lote_092024.groupby(['Amostra'])['GBM_mercado',"Renewal Demand"].describe().T

In [ ]:
#df_lote_092024.columns.to_list()

In [ ]:
df_lote_escorar=df_lote_092024.copy()

df_lote_escorar=df_lote_092024[['Premio_oferecido_com_batente','Pr_anterior_TI','Premio_IC100','GBM_mercado',
                                          'Classe_bonus',"MD_sexo","cod_corr","cod_end","MD_tempo_casa_2",
                                          "per_com", "MD_menor26","UF","Renovacao_ant","fx_ir_12m","is_casco_ofer",
                                          "MD_fator_ajuste","md_capac","MD_categoria",
                                          "MD_comb","MD_qtd_assis_cpf", "idade_ult_RNS","canal","MD_tipo_renov",'FABRICA',
                                          'MD_est_civil','Grupo_veiculo_new']]

In [ ]:
#print(df_lote_escorar['classe_bonus'].value_counts(ascending=False))
#df_lote_escorar['Classe_bonus'].describe()

premio|Premio_oferecido_com_batente
is_casco_ofer|is_casco_ofer
GBM_mercado|GBM_mercado
Premio_IC100|Premio_IC100
premio_anterior|Pr_anterior_TI
canal_bin| canal
Cod_End_bin|  cod_end
fator_ajuste_bin|  MD_fator_ajuste
per_com|per_com
MD_tempo_casa_2|MD_tempo_casa_2
idade_ult_RNS_bin|idade_ult_RNS
MD_comb_bin|MD_comb
UF_bin      |     UF
fx_ir_12m_bin|    fx_ir_12m
MD_tipo_renov,|   MD_tipo_renov

In [ ]:
df_lote_escorar["premio"] = df_lote_escorar["Premio_oferecido_com_batente"]

In [ ]:
df_lote_escorar.rename(columns={'Pr_anterior_TI':'premio_anterior', 
                                   'cod_end': 'Cod_End',
                                   'md_capac': 'MD_capac',
                                   'Classe_bonus':'classe_bonus',
                                   'fator_ajuste': 'MD_fator_ajuste'},inplace=True)

In [ ]:
df_lote_escorar=df_lote_escorar[df_lote_escorar['UF']!= "99"]

In [ ]:
df_lote_escorar.shape

In [ ]:
df_lote = categorizar(df_lote_escorar)

In [ ]:
df_lote["prob_est"]=model.predict(df_lote)

In [ ]:
df_lote.groupby(["Amostra"])["prob_est" , "Renewal Demand"].describe().T

## Otimização Teste 

In [ ]:
print(df_lote['Amostra'].value_counts(ascending=False))

In [ ]:
#df_lote_otim=df_mod.copy()
df_lote_otim_atu = df_lote.query('Amostra == "Otimização Atu"')
df_lote_otim_atu.shape

In [ ]:
# Plot the data to visualize the relationship between price and quantity
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_lote_otim_atu, x='premio', y='prob_est')
plt.title('Preço vs Retenção')
plt.xlabel('Preço')
plt.ylabel('Retenção')
plt.show()

In [ ]:
df_lote_otim = df_lote_otim_atu

In [ ]:
#x = np.linspace(df_train['premio'].min(),2*df_train['premio'].max(),df_train.shape[0])
#len(x)

In [ ]:
#model_elast.params[1]

In [ ]:
df_lote_otim = df_lote_otim_atu.sample(n = 100) 

In [ ]:
def demand(premio):
    df_lote_otim["premio"]=premio
    quantity= model.predict(df_lote_otim)
    return quantity


#Define the revenue function
def revenue(premio):
    df_lote_otim["premio"]=premio
    revenue=premio * demand(premio)
    return revenue

def negrevenue(premio):
    df_lote_otim["premio"]=premio
    return -revenue(premio)

In [ ]:
#teste  = negrevenue(df_lote_otim["Premio_oferecido_com_batente"])


In [ ]:
#quantity = demand(df_lote_otim["Premio_oferecido_com_batente"])
#revenue = revenue(df_lote_otim["Premio_oferecido_com_batente"])
#premio = df_lote_otim["Premio_oferecido_com_batente"]


In [ ]:
df  = pd.DataFrame({"quantity": pd(quantity),
                    "revenue": pd.Series(revenue),
                    "premio_in": pd.Series(premio_in),
                    "negrevenue": pd.Series(premio_in)})
df

In [ ]:
df_lote_otim = df_lote_otim_atu.sample(n = 500) 

In [ ]:
#df_lote_otim = df_otim1
df_lote_otim.shape

In [ ]:
df_lote_otim["Margem Inicial"] = df_lote_otim["prob_est"]*(df_lote_otim["Premio_oferecido_com_batente"]-df_lote_otim["Act_cost"])

In [ ]:
print('Premio Inicial: %2f' % np.mean(df_lote_otim["premio"]))
print('Probabilidade Inicial: %2f' % np.mean(df_lote_otim["prob_est"]))
print('Margem Inicial: %2f' % np.mean(df_lote_otim["Margem Inicial"]))

#########################################

In [ ]:
%%time

def demand(premio):
    df_lote_otim["premio"]=premio
    demanda= model.predict(df_lote_otim)
    return demanda

def revenue(premio):
    df_lote_otim["premio"]=premio
    return premio * demand(premio)


def negrevenue(premio,demanda_alvo):
    df_lote_otim["premio"]=premio
    df_lote_otim["demanda_alvo"]=demanda_alvo
    margem =  revenue(premio)
    return -np.sum(margem)

# restricão demanda alvo
def constraint(premio,demanda_alvo):
    df_lote_otim["premio"]=premio
    demanda= model.predict(df_lote_otim)
    demanda_alvo = df_lote_otim["demanda_alvo"]
    return  np.sum(demanda) - np.sum(demanda_alvo)


def constraint2(premio,demanda_alvo):
    margem_inicial = df_lote_otim["prob_est"]*(df_lote_otim["Premio_oferecido_com_batente"]-df_lote_otim["Act_cost"])
    df_lote_otim["premio"]=premio
    df_lote_otim["demanda"] = model.predict(df_lote_otim)
    demanda_alvo = df_lote_otim["demanda_alvo"]
    margem_otim = df_lote_otim["demanda"]*(df_lote_otim["premio"]-df_lote_otim["Act_cost"])
    return  np.sum(margem_otim) - np.sum(margem_inicial)


def constraint3(premio,demanda_alvo):
    lb = df_lote_otim["Premio_oferecido_com_batente"]*0.90
    return lb

def constraint4(premio,demanda_alvo):
    ub = df_lote_otim["Premio_oferecido_com_batente"]*1.1
    return ub
 
restr_demanda = {'type': 'eq', 'fun': constraint}   
restr_margem  = {'type': 'ineq', 'fun': constraint2} 
restr_lb = {'type': 'ineq', 'fun': constraint3}   
restr_ub  = {'type': 'ineq', 'fun': constraint4} 
 
#restricoes = ([restr_demanda,restr_margem,restr_lb,restr_ub])     

restricoes = ([restr_demanda,restr_margem])    

demanda_base_case = np.round(np.mean(df_lote["prob_est"]),2)

demanda_alvo=0.5

premio_inicial = df_lote_otim["Premio_oferecido_com_batente"]*(demanda_base_case/demanda_alvo)

# limitando o premio fique dentro do range
bounds = [(0.9 * premio_base, premio_base*1.1) for premio_base in df_lote_otim["Premio_oferecido_com_batente"]]

# rodando a otimizaçao
result = minimize(negrevenue,
                  premio_inicial,
                  args=(demanda_alvo), 
                  method = 'L-BFGS-B', 
                  tol=0.01,
                  constraints=restricoes,
                  bounds=bounds)

In [ ]:
# premio otimizados
premio_otim = result.x
df_lote_otim["premio"]=result.x

df_lote_otim["demanda otimizada"] = model.predict(df_lote_otim)
df_lote_otim["Margem otimizada"] = df_lote_otim["demanda otimizada"]*(df_lote_otim["premio"]-df_lote_otim["Act_cost"])

print(np.mean(df_lote_otim["Premio_oferecido_com_batente"]))
print(np.mean(df_lote_otim["premio"]))
print(np.mean(df_lote_otim["Margem Inicial"]))
print(np.mean(df_lote_otim["Margem otimizada"]))
print(np.mean(df_lote_otim["prob_est"]))
print(np.mean(df_lote_otim["demanda otimizada"]))

In [ ]:
%%time

def margem_esperada(premio,demanda_alvo):
    df_lote_otim["premio"]=premio
    demanda= model.predict(df_lote_otim)
    df_lote_otim["demanda_alvo"]=demanda_alvo
    custo= df_lote_otim["Act_cost"]
    #margem_esperada = demanda *np.exp(-0.0005*premio) * (premio-custo)
    margem_esperada = demanda *np.exp(-0.0005*premio) * (premio-custo)
    margem_total = -np.sum(margem_esperada) 
    return margem_total

# restricão demanda alvo
def constraint(premio,demanda_alvo):
    df_lote_otim["premio"]=premio
    demanda= model.predict(df_lote_otim)
    demanda_alvo = df_lote_otim["demanda_alvo"]
    return  np.sum(demanda) - np.sum(demanda_alvo)


def constraint2(premio,demanda_alvo):
    margem_inicial = df_lote_otim["prob_est"]*(df_lote_otim["Premio_oferecido_com_batente"]-df_lote_otim["Act_cost"])
    df_lote_otim["premio"]=premio
    df_lote_otim["demanda"] = model.predict(df_lote_otim)
    demanda_alvo = df_lote_otim["demanda_alvo"]
    margem_otim = df_lote_otim["demanda"]*(df_lote_otim["premio"]-df_lote_otim["Act_cost"])
    return  np.sum(margem_otim) - np.sum(margem_inicial)


def constraint3(premio,demanda_alvo):
    lb = df_lote_otim["Premio_oferecido_com_batente"]*0.90
    return lb

def constraint4(premio,demanda_alvo):
    ub = df_lote_otim["Premio_oferecido_com_batente"]*1.1
    return ub
 
restr_demanda = {'type': 'eq', 'fun': constraint}   
restr_margem  = {'type': 'ineq', 'fun': constraint2} 
restr_lb = {'type': 'ineq', 'fun': constraint3}   
restr_ub  = {'type': 'ineq', 'fun': constraint4} 
 
#restricoes = ([restr_demanda,restr_margem,restr_lb,restr_ub])     

restricoes = ([restr_demanda,restr_margem])    

demanda_base_case = np.round(np.mean(df_lote["prob_est"]),2)

demanda_alvo=0.55

#remio_inicial = df_lote_otim["Premio_oferecido_com_batente"]*(demanda_base_case/demanda_alvo)
premio_inicial = df_lote_otim["Premio_oferecido_com_batente"]
# limitando o premio fique dentro do range
bounds = [(0.9 * premio_base, premio_base*1.1) for premio_base in df_lote_otim["Premio_oferecido_com_batente"]]

# rodando a otimizaçao
result = minimize(margem_esperada,
                  premio_inicial,
                  args=(demanda_alvo), 
                  method = 'L-BFGS-B', 
                  tol=0.01,
                  constraints=restricoes,
                  bounds=bounds)



In [ ]:
print(result.x)

In [ ]:
print(result.fun)

In [ ]:
# premio otimizados
premio_otim = result.x
df_lote_otim["premio"]=result.x

df_lote_otim["demanda otimizada"] = model.predict(df_lote_otim)
df_lote_otim["Margem otimizada"] = df_lote_otim["demanda otimizada"]*(df_lote_otim["premio"]-df_lote_otim["Act_cost"])

In [ ]:
print(np.mean(df_lote_otim["Premio_oferecido_com_batente"]))
print(np.mean(df_lote_otim["premio"]))
print(np.mean(df_lote_otim["Margem Inicial"]))
print(np.mean(df_lote_otim["Margem otimizada"]))
print(np.mean(df_lote_otim["prob_est"]))
print(np.mean(df_lote_otim["demanda otimizada"]))


In [ ]:
prices_df = pd.DataFrame({
    'chave': df_lote_otim["chave"],
    'Act_cost':  df_lote_otim["Act_cost"],
    'Premio ofertado com batente':  df_lote_otim["Premio_oferecido_com_batente"],
    'Optimized Prices': df_lote_otim["premio"],
    'lb_bound' : [b[0] for b in bounds],
    'up_bound' : [b[1] for b in bounds],
    'prob_est':         df_lote_otim["prob_est"],
    'renewal demand':df_lote_otim["demanda otimizada"],
    'variação':df_lote_otim["premio"]/df_lote_otim["Premio_oferecido_com_batente"]
})
prices_df.head(20)

In [ ]:
df_plot = df_lote_otim[["prob_est","premio","Amostra"]].copy()
df_plot.dtypes


In [ ]:
df_plot.groupby(['Amostra'])['prob_est'].mean()

In [ ]:
# teste
teste = df_plot.groupby(['Amostra'], as_index=False).agg({'prob_est':['mean','std']})
teste


In [ ]:
#df_plot["range_preco"] = list(range(np.min(df_plot["premio"].astype('int')), np.max(df_plot["premio"].astype('int')), 500))



In [ ]:
#plt.figure(figsize=(9,6))
#plt.plot(preco, np.mean(prices_df['renewal demand']), color='grey')
#plt.xlabel('Preço')
#plt.ylabel('Renovação');
#plt.title('Preço Renov');

In [ ]:
prices_df["variação"].describe()

#criando a função

In [ ]:
df_relatorio = df_lote_otim.copy()
df_relatorio

In [ ]:
df_relatorio["per_com_pct"] = df_relatorio['per_com']/100
df_relatorio["premio_prob"] = df_relatorio["demanda otimizada"]*df_relatorio["premio"]
df_relatorio["Act_cost_prob"] = df_relatorio["demanda otimizada"]*df_relatorio["Act_cost"]                                                                
df_relatorio["comissao_valor"] = df_relatorio["per_com_pct"] *df_relatorio["premio"]
df_relatorio["comissao_valor_prob"] = df_relatorio["per_com_pct"] *df_relatorio["premio_prob"]


In [ ]:
apolice_ofertada  = np.round(df_relatorio.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio["demanda otimizada"]),0)
premio_ofertado   = np.mean(df_relatorio["premio"])
retencao_estimada = np.mean(df_relatorio["demanda otimizada"])
retencao_base_case = np.mean(df_relatorio["prob_est"])
premio_esperado   = np.sum(df_relatorio["premio_prob"])/np.sum(df_relatorio["demanda otimizada"])
risco_ofertado    = np.mean(df_relatorio["Act_cost"])
risco_esperado    = np.sum(df_relatorio["Act_cost_prob"])/np.sum(df_relatorio["demanda otimizada"])
sp_ofertado       = np.sum(df_relatorio["Act_cost"])/np.sum(df_relatorio["premio"])
sp_esperado       = np.sum(df_relatorio["Act_cost_prob"])/np.sum(df_relatorio["premio_prob"])
comis_ofertado    = np.sum(df_relatorio["comissao_valor"])/np.sum(df_relatorio["premio"])
comis_esperado    = np.sum(df_relatorio["comissao_valor_prob"])/np.sum(df_relatorio["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749

In [ ]:
print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_baseCase : %s' % np.round(retencao_base_case*100,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))
print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

In [ ]:
def maxProfit(prices):
    dif=[]
    current_min=[]
    for i in range(len(prices)):
        for k in range(1,len(prices)):
            dif.append(prices[i]-prices[k])
            if prices[i]-prices[k]==min(dif):
                if i<k:
                    current_min.append([i,k])
    print("current indices of price array where profit is maximum:" +str(current_min))
    
    res=prices[current_min[-1][1]]-prices[current_min[-1][0]]
    if res<0:
        return str(0) + "\n"
    elif res>=0:
        return "Max profit is " +str(res) + "\n"
    
  

In [ ]:
print(maxProfit([7,1,5,3,6,4]))

In [ ]:
          if i<k:                                # Is the trade valid?
            dif.append(prices[i]-prices[k])    # Update the dif and 
            if prices[i]-prices[k]==min(dif):  # current_min arrays.
                current_min.append([i,k])
                

print(maxProfit([7,6,4,3,1]))
print(maxProfit([3,3,5,0,0,3,1,4]))
print(maxProfit([3,3,0,3,1,4]))